In [1]:
from functools import partial
import numpy as np
import PyQt5
import qcodes
from qcodes.data.io import DiskIO
from qcodes.data.data_set import DataSet


from core_tools.GUI.keysight_videomaps.liveplotting import liveplotting, set_data_saver
from core_tools.GUI.keysight_videomaps.data_saver.qcodes import QCodesDataSaver
from core_tools.GUI.keysight_videomaps.data_getter.scan_generator_OPX import fake_digitizer
from core_tools.GUI.qt_util import qt_init

from pulse_lib.base_pulse import pulselib

from core_tools.drivers.gates import gates as Gates
from core_tools.drivers.hardware.hardware import hardware as Hardware

from core_tools.GUI.virt_gate_matrix_qml.gui_controller import (
    virt_gate_matrix_GUI as VirtGateMatrixGUI,
)
from core_tools.GUI.param_viewer.param_viewer_GUI_main import (
    param_viewer as ParamViewerGUI,
)

# QDACII driver
from qcodes_contrib_drivers.drivers.QDevil import QDAC2
# OPX driver
from qualang_tools.external_frameworks.qcodes.opx_driver import OPX

station = qcodes.Station()

# host: router IP address, port: 11+ last 3 dig of OPX
#opx = OPX(config, name="OPX_alice", host="opx-sc-001", port='11253', close_other_machines=False)
# opx = OPX(config, name="OPX_alice", host="opx-sc-001", port='11253', close_other_machines=True)
# #Add the OPX instrument to the QCoDeS station
# station.add_component(opx)


/tmp/ipykernel_628177/2571828664.py:5: QCoDeSDeprecationWarning: The qcodes.data module is deprecated. Use "qcodes_loop.data" as an alternative.
  from qcodes.data.io import DiskIO
/tmp/ipykernel_628177/2571828664.py:5: QCoDeSDeprecationWarning: The qcodes.data.io module is deprecated. Use "qcodes_loop.data.io" as an alternative.
  from qcodes.data.io import DiskIO
/tmp/ipykernel_628177/2571828664.py:6: QCoDeSDeprecationWarning: The qcodes.data.data_set module is deprecated. Use "qcodes_loop.data.data_set" as an alternative.
  from qcodes.data.data_set import DataSet
/local/work/masker/core_tools/core_tools/GUI/keysight_videomaps/data_saver/qcodes.py:4: QCoDeSDeprecationWarning: The qcodes.measure module is deprecated. Use "qcodes_loop.measure" as an alternative.
  from qcodes.measure import Measure


2024-02-13 08:24:33,589 - qm - INFO     - Starting session: b38af719-ee5c-493e-b9a4-0eccabf51668


In [2]:
#print(f'python version = {sys.version}')
from qm import QuantumMachinesManager
qmm = QuantumMachinesManager(host='opx-sc-001', cluster_name="Cluster_1") # IP address of router
qlst = qmm.list_open_quantum_machines()
print(f'{qlst=}')
for q in qlst:
    qm = qmm.get_qm(q)
    qm.close()


2024-02-13 08:38:26,754 - qm - INFO     - Starting session: 90b8d39a-f65e-4840-a7fb-ed3d6a10bc66
2024-02-13 08:38:27,888 - qm - INFO     - Performing health check
2024-02-13 08:38:27,893 - qm - INFO     - Health check passed
qlst=['qm-1707180470608']


In [22]:
import numpy as np

AWG_VirtGateMatrix = [
    # Vgs13    Vgs11    Vgs09    Vds    Vcm]
    [  1,        0,       0,      0,      1],   # HG13b
    [  0,        1,       0,      0,      1],   # HG11b
    [  0,        0,       1,      0,      1],   # HG09b
    [  0,        0,       0,    +0.5,     1],   # HG07b
    [  0,        0,       0,    -0.5,     1],   # HG12b
] 
m = AWG_VirtGateMatrix
minv = np.linalg.inv(AWG_VirtGateMatrix)
print(minv)
np.matmul(m, [23,45,67,10,1000])

[[ 1.   0.   0.  -0.5 -0.5]
 [ 0.   1.   0.  -0.5 -0.5]
 [ 0.   0.   1.  -0.5 -0.5]
 [ 0.   0.   0.   1.  -1. ]
 [ 0.   0.   0.   0.5  0.5]]


array([1023., 1045., 1067., 1005.,  995.])

In [2]:

qdac2_addr = 'qdac-sc-001'
qdac2 = QDAC2.QDac2('QDAC', visalib='@py', address=f'TCPIP::{qdac2_addr}::5025::SOCKET')
# Add the QDAC2 instrument to the QCoDeS station
station.add_component(qdac2)


Connected to: QDevil QDAC-II (serial:146, firmware:7-0.17.5) in 0.17s


'QDAC'

In [ ]:
# for i in range(1, 1+24):

i = 10
qdac2.add_parameter(
            f'dac{i}',
            qcodes.DelegateParameter,
            source=qdac2.channel(i).dc_constant_V,
            label=f'dac{i}',
            unit='V',
            scale=0.001
        )


In [ ]:



qcodes.DelegateParameter('dac10',  qdac2.ch10.dc_constant_V, instrument=qdac2 )

In [ ]:
qdac2.ch15.dc_constant_V.set(0.1)
qdac2.ch15.dc_constant_V()

In [ ]:
qdac2.dac10.set(1000)


In [ ]:

qdac2.dac10()

In [6]:
qdac2.dac10()

9.7999811

In [3]:
hw = Hardware("hardware")


In [4]:

hw.dac_gate_map = {
    "VGS09_15": (0, 15),        
}


In [ ]:


# For each of the gates create a DelegateParameter to the underlying qdac2 channel function
for gat, gmap in hw.dac_gate_map.items():
    cnum = gmap[1]
    dac_n = f'dac{cnum:d}'
    fstr = f'ch{cnum:d}'
    func = getattr(qdac2, fstr)
    func = getattr(func, 'dc_constant_V' )
    qcodes.DelegateParameter(dac_n,  func )

#p = qc.Instrument._all_instruments
pppp = qcodes.Instrument
print(pppp)



In [3]:
pppp = qcodes.Instrument
print(pppp)

<class 'qcodes.instrument.instrument.Instrument'>


In [ ]:
qcodes.DelegateParameter('dac15',  gates. )

In [5]:
gates = Gates("gates", hardware=hw, dac_sources=[qdac2], dc_gain={})

In [9]:
gates.VGS09_15.set(0.11)

KeyError: ('dac15', 'setting gates_VGS09_15 to 0.11')